[habr](https://habrahabr.ru/company/ods/blog/326418/)

# Стохастический градиентный спуск (SGD, Stochastic Gradient Descent)


Вектор градиента (состоящий из частных производных функции) задаёт направление наискорейшего возрастания функции

Стохастический -- на каждом шаге считаем градиент не по всей выборке, а только по N объектов. Каждая итерация считается мгновенно, но их потребуется больше. Зато можно не держать датасет в памяти, а идти по нему последовательно.

Важный факт: стохастический градиентный бустинг сходится в результате к тому же результату, что и обычный.

# Работа с категориальными признаками: 
(Label Encoding, One-Hot Encoding, Hashing trick)

## Label Encoding
`sklearn.preprocessing.LabelEncoder`

* нужна уверенность, что трейн покрывает все возможные значения признаков
* создает (неверное) евклидовое пространство -- ни линейные модели, ни ближайшие соседи адекватны не будут

## One-hot encoding
`sklearn.preprocessing.OneHotEncoder`
* уже более осмысленно для линейных моделей
* раздувает выборку (по умолчанию возвращает разреженную матрицу)

## Хэширование признаков (Hashing trick)
Аналогично label encoding, но нормально работает с новыми метками. Правда, могут быть коллизии

In [4]:
import pandas as pd

hash_space = 10
hashing_example = pd.DataFrame([{i: 0.0 for i in range(hash_space)}])
for s in ('job=student', 'marital=single', 'day_of_week=mon'):
    print(s, '->', hash(s) % hash_space)
    hashing_example.loc[0, hash(s) % hash_space] = 1
hashing_example

job=student -> 3
marital=single -> 6
day_of_week=mon -> 1


,0,1,2,3,4,5,6,7,8,9
0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


**NB**: хэшируем вместе с названием переменной, чтобы не смешивались одинаковые значения для разных переменных

Подробней про хэширование признаков (learning to hash) можно почитать в [этом](https://arxiv.org/abs/1509.05472) обзоре, а также в [материалах](https://github.com/esokolov/ml-course-hse/blob/master/2016-fall/lecture-notes/lecture06-linclass.pdf) Евгения Соколова.

# Библиотека Vowpal Wabbit
Хорошо подходит для онлайн-обучения, есть реализация хэширования признаков

Формат:
```[Label] [Importance] [Tag]|Namespace Features |Namespace Features ... |Namespace Features

Namespace=String[:Value]

Features=(String[:Value] )*
```
где [] обозначает необязательные элементы, а (...)* означает повтор неопределенное число раз.

* Label является числом, "правильным" ответом. В случае классификации обычно принимает значение 1/-1, а в случае регрессии некоторое вещественное число
* Importance является числом и отвечает за вес примера при обучении. Это позволяет бороться с проблемой несбалансированных данных, изученной нами ранее
* Tag является некоторой строкой без пробелов и отвечает за некоторое "название" примера, которое сохраняется при предсказании ответа. Для того, чтобы отделить Tag от Importance лучше начинать Tag с символа '.
* Namespace служит для создания отдельных пространств признаков. В аргументах Namespace именуются по первой букве, это нужно учитывать при выборе их названий
* Features являются непосредственно признаками объекта внутри Namespace. Признаки по умолчанию имеют вес 1.0, но его можно переопределить, к примеру feature:0.1.
